### CSV 파일읽기

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df_exchanges = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/98_crawling_scraping/DEXKOUS.csv')

|키워드 매개변수(params)|설명|
|-----|-----|
|encoding|파일의 인코딩을 지정한다|
|header|헤더로 사용할 줄 번호(0부터 시작)을 지정, 이전의 줄은 무시, None값을 plug-in 하면 가장 윗 row부터 counting|
|names|column names 지정|
|skipinitialspace|True라면 쉼표 뒤의 공백 무시|
|index_col|인덱스로 사용할 열의 번호 지정|
|parse_dates|True로 지정시 인덱스에 사용한 열이 날짜일 경우 파싱시도, 날짜형식은 알아서 추출|
|date_parser|날짜를 파싱할 함수를 지정|
|na_values|NaN으로 간주할 문자열 목록 지정|

#### 엑셀 파일 읽기


경제 활동 인구 총괄 엑셀 파일을 다운로드하고, 다운로드 이후의 실업률을 추출해보자. 엑셀의 종류는 크게 2가지로 존재한다.

    * 엑셀 03 버전 이전의 .xls 파일

    * 엑셀 07 이후부터의 표준화된 .xlsx 파일(오피스 오픈 XML로 표준화된 오픈 형식)

pandas에서 pd.read_excel()을 활용할 수 있으나, 파이썬으로 엑셀을 읽는 라이브러리로는 xlrd가 유명하다. 

pandas의 pd.read_excel()함수 또한 xlrd를 활용하지만, xlrd 패키지가 설치된 상태가 아니므로 활용을 원한다면 pip install xlrd를 통해 설치 후 활용하면 된다.

In [ ]:
!pip install xlrd

In [ ]:
## kosis 데이터 참고 링크
# 주제별 통계[고용, 노동, 임금]
# [고용] -> [경제활동인구조사] -> [경제활동인구총괄(구직기간 1주기준, 63년~2014년 12월)]
# -> [연령별 경제활동인구 총괄(구직기간 1주 기준)]을 클릭

# https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_1D07002S&vw_cd=MT_ZTITLE&list_id=B13&scrId=&seqNo=&lang_mode=ko&obj_var_id=&itm_id=&conn_path=MT_ZTITLE&path=%252FstatisticsList%252FstatisticsListIndex.do

In [ ]:
import pandas as pd
df_jobs = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/98_crawling_scraping/guzik.xlsx')

In [ ]:
df_jobs.head(10)

In [ ]:
## 가장 위의 header를 첫번째 행의 값과 교환
# header를 변경

# df에 존재하는 컬럼들을
# 변수에 할당
df_jobs_col = df_jobs.columns
df_jobs_col

In [ ]:

df_jobs.rename(columns = df_jobs.iloc[0], inplace = True)
df_jobs

In [ ]:
# 행과 열의 위치를 바꾸는 방법은 transpose도 있지만
# stack 함수 또한 존재합니다. # 결과는 series 
# 2030 세대의 실업률을 보고 싶다면?

df_jobs.loc[1] = df_jobs_col
df_jobs_renewal = df_jobs[1:].copy()

In [ ]:
df_jobs_renewal = df_jobs_renewal.reset_index(drop = True)

In [ ]:
# 결과는 series이며 
# 2030세대를 보고 싶다면
# 아래의 인덱싱을 2와 3의 고정값으로 지정해주어야 한다
df_jobs_renewal_20s = df_jobs_renewal.stack()[2]
df_jobs_renewal_30s = df_jobs_renewal.stack()[3]

## Read한 데이터 기반의 Graph 작성

In [ ]:
df_jobs_renewal_20s

In [ ]:
# History Data

from datetime import datetime
import pandas as pd

import matplotlib
matplotlib.rcParams['font.sans-serif'] = 'NanumGothic,AppleGothic'
import matplotlib.pyplot as plt


#1981~2014 사이의 환율 및 고용률 출력
# Read Exchange Info
df_exchange = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/98_crawling_scraping/DEXKOUS.csv',
                          header=1, names=['DATE','DEXKOUS'], skipinitialspace=True, index_col=0)

df_exchange

In [ ]:
# 위의 데이터를 년도로만 축약된 데이터로 표현

years = {}
output = []

for index in df_exchange.index:
    year = int(index.split('-')[0])
    if (year not in years) and (1981 < year < 2014):
        if df_exchange.DEXKOUS[index] != ".":
            years[year] = True
            output.append([year, float(df_exchange.DEXKOUS[index])])

df_exchange_pre = pd.DataFrame(output)

In [ ]:
## 고용률 통계 구하기
df_jobs = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/98_crawling_scraping/guzik.xlsx')
result_jobs = []

# stack을 통한 행//열 전환
# 20대의 데이터 통계 확인
df_jobs_pre = df_jobs.stack()[3]

for index in df_jobs_pre.index:
    try:
        if 1981 <= int(index) <= 2014:
            result_jobs.append([int(index), float(df_jobs_pre[index])])

    except:
        pass

s_jobs = pd.DataFrame(result_jobs)

In [ ]:
# 그래프 작성
plt.subplot(2,1,1) 
plt.plot(df_exchange_pre[0],df_exchange_pre[1], label='won/dollar')
plt.xlim(1981,2014)
plt.ylim(500,2500)
plt.show()

# 두번째 그래프 작성
print(s_jobs)
plt.subplot(2,1,2)
plt.plot(s_jobs[0], s_jobs[1], label='Emplotment Rate(%)')
plt.xlim(1981,2014)
plt.ylim(5000,10000)
plt.legend(loc = 'best')

## OPEN API 활용

열린 데이터(이하 Open Data)는 정부 및 지자체 등이 보유하고 있는 데이터를 공개하여 자유롭게 활용하도록 하는 것이다. 

미국 정부와 영국 정부는 이러한 Open Data의 선구자로 유명하며, 한국도 조금씩 Open Data가 많이 공유되고 있다. 우리나라의 대표적인 Open Data는 아래와 같다.


|Web Site|Description|
|---|---|
|공공데이터포털(https://www.data.go.kr/)|국내의 Open Data가 대부분 통합된 사이트 이다. 다양한 데이터가 공개되어 있다.|
|서울데이터 열린광장(https://data.seoul.go.kr/)|서울에서 제공하는 Open Data이다.|
|기상청 날씨마루(https://bd.kma.go.kr/kma2020/svc/main.do)|기상청에서 제공하는 날씨데이터이다.|

## * 저작권 표시 및 동일조건 변경 허락의 조건에서

위의 Open Data는 모든 사람이 자유롭게 사용 및 재사용이 가능하며 재배포할 수 있는 데이터이다

In [ ]:
# end of file